<a href="https://colab.research.google.com/github/NikNord174/Autoencoder/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoder

In [7]:
import os
import sys

from time import time

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Compose, ToTensor, Normalize


In [8]:
BATCH_SIZE = 64
NUM_WORKERS = 2
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')


In [9]:
transform=transforms.Compose([
                              ToTensor(),
                              Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ])
train = datasets.CIFAR10('../data',
                         train=True,
                         download=True,
                         transform=transform)
test = datasets.CIFAR10('../data',
                        train=False,
                        download=True,
                        transform=transform)
train_loader = torch.utils.data.DataLoader(train,
                                           batch_size=BATCH_SIZE,
                                           num_workers=NUM_WORKERS,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test,
                                          batch_size=BATCH_SIZE,
                                          num_workers=NUM_WORKERS,
                                          shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [11]:
len(test_loader)

157

## Models

### Test cells

In [6]:
def simple_conv_block(decoder: bool = False,
                      input_channels: int = 3,
                      output_channels: int = 3,
                      kernel_size: int = 3,
                      stride: int = 1,
                      padding: int = 0,
                      final_layer: bool = False) -> nn.Sequential:
    if not decoder:
      if not final_layer:
        return nn.Sequential(
            nn.Conv2d(in_channels=input_channels,
                      out_channels=output_channels,
                      kernel_size=kernel_size,
                      stride=stride,
                      padding=padding),
            nn.BatchNorm2d(output_channels),
            nn.MaxPool2d(kernel_size=3, stride=None, padding=0),
            #nn.Dropout(p=0.2),
            nn.LeakyReLU(0.2)
            )
      else:  # Final Layer
        return nn.Sequential(
            nn.Conv2d(in_channels=input_channels,
                      out_channels=output_channels,
                      kernel_size=kernel_size,
                      padding=padding)
            )
    else:
        if not final_layer:
                return nn.Sequential(
                    nn.ConvTranspose2d(in_channels=input_channels,
                              out_channels=output_channels,
                              kernel_size=kernel_size,
                              padding=padding),
                    nn.BatchNorm2d(output_channels),
                    #nn.Dropout(p=0.2),
                    nn.LeakyReLU(0.2)
                )
        else:
                return nn.Sequential(
                    nn.ConvTranspose2d(in_channels=input_channels,
                              out_channels=output_channels,
                              kernel_size=kernel_size,
                              padding=padding)
                )
        
encoder = nn.Sequential(simple_conv_block(input_channels=3,
                                          output_channels=30),
                        simple_conv_block(input_channels=30,
                                          output_channels=60),
                        simple_conv_block(input_channels=60,
                                          output_channels=120,
                                          final_layer=True),
                        simple_conv_block(input_channels=120,
                                          output_channels=240,
                                          final_layer=True),
        )

fake = torch.randn(64, 3, 32, 32)
m = simple_conv_block(input_channels=3, output_channels=30)
#m(fake).shape
encoder(fake).shape

RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [5]:
fake = torch.randn(64, 3, 32, 32)
m = nn.Conv2d(3, 30, 3, 3)
a = m(fake)
m_1 = nn.ConvTranspose2d(60, 3, 3, 1,)
b = m_1(a)
print(a.shape)
print(b.shape)
#fake = torch.randn(64, 3, 32, 32).to(DEVICE)
#model(fake).shape

torch.Size([64, 60, 10, 10])
torch.Size([64, 3, 12, 12])


In [ ]:
layers = []
channels = [3, 30, 120]#, 240]
for i in range(len(channels)-1):
    if channels[i] != channels[-2]:
        layers.append(simple_conv_block(input_channels=channels[i],
                                             output_channels=channels[i+1]))
    else:
        layers.append(simple_conv_block(input_channels=channels[i],
                                             output_channels=channels[i+1],
                                             final_layer=True))
encoder = nn.Sequential(*layers)
fake = torch.randn(64, 3, 32, 32)
print(fake.shape)
#flatten = nn.Flatten()
#fake=flatten(fake)
#print(fake.shape)
#print(encoder)
encoder(fake).shape

### Model

In [12]:
from models import Autoencoder
model = Autoencoder()

In [14]:
print(model)

Autoencoder(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 60, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.2, inplace=False)
      (3): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Conv2d(60, 120, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.2, inplace=False)
      (3): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Conv2d(120, 240, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    )
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=3840, out_features=300, bias=True)
  (rev_linear): Linear(in_features=300, out_features=3840, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(240, 120, kernel_size=(4, 4), stride=(2, 2

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.channels = [3, 60, 120, 240]
        self.hidden_state = 300
        self.encoder = self.encoder_layers()
        self.flatten = nn.Flatten()
        self.lin_neurons = [240, 4, 4]
        self.enc_neurons = np.prod(self.lin_neurons)
        self.linear = nn.Linear(self.enc_neurons,
                                self.hidden_state)
        self.rev_linear = nn.Linear(self.hidden_state,
                                    self.enc_neurons)
        self.decoder = self.decoder_layers()


    def simple_enc_block(self,
                         input_channels: int = 3,
                         output_channels: int = 3,
                         kernel_size: int = 4,
                         stride: int = 2,
                         padding: int = 1,
                         final_layer: bool = False) -> nn.Sequential:
        if not final_layer:
            return nn.Sequential(
                nn.Conv2d(in_channels=input_channels,
                          out_channels=output_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding),
                nn.BatchNorm2d(output_channels),
                #nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
                nn.Dropout(p=0.2),
                nn.LeakyReLU(0.2)
                )
        else:
            return nn.Sequential(
                nn.Conv2d(in_channels=input_channels,
                          out_channels=output_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding)
                )


    def encoder_layers(self) -> nn.Sequential:
        layers = []
        for i in range(len(self.channels)-1):
            if self.channels[i] != self.channels[-2]:
                layers.append(self.simple_enc_block(input_channels=self.channels[i],
                                                     output_channels=self.channels[i+1]))
            else:
                layers.append(self.simple_enc_block(input_channels=self.channels[i],
                                                     output_channels=self.channels[i+1],
                                                     final_layer=True))
        return nn.Sequential(*layers)
    
    
    def simple_dec_block(self,
                         input_channels: int = 3,
                         output_channels: int = 3,
                         kernel_size: int = 4,
                         stride: int = 2,
                         padding: int = 1,
                         final_layer: bool = False) -> nn.Sequential:
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels=input_channels,
                                   out_channels=output_channels,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding),
                nn.BatchNorm2d(output_channels),
                nn.Dropout(p=0.2),
                nn.LeakyReLU(0.2)
                )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels=input_channels,
                                   out_channels=output_channels,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding)
                )


    def decoder_layers(self) -> nn.Sequential:
        layers = []
        dec_channels = list(reversed(self.channels))
        for i in range(len(dec_channels)-1):
            if dec_channels[i] != dec_channels[-2]:
                layers.append(self.simple_dec_block(input_channels=dec_channels[i],
                                                    output_channels=dec_channels[i+1]))
            else:
                layers.append(self.simple_dec_block(input_channels=dec_channels[i],
                                                    output_channels=dec_channels[i+1],
                                                    final_layer=True))
        return nn.Sequential(*layers)


    def forward(self, x):
        encoded = self.encoder(x)
        hidden_layer = torch.flatten(encoded, start_dim = 1)
        hidden_layer = self.linear(hidden_layer)
        hidden_layer = self.rev_linear(hidden_layer)
        hidden_layer = hidden_layer.view(-1, *self.lin_neurons)
        decoded = self.decoder(hidden_layer)
        return decoded
    

model = Autoencoder().to(DEVICE)

In [ ]:
fake = torch.randn(64, 3, 32, 32).to(DEVICE)
print(fake.shape)
print(model(fake).shape)
a = Autoencoder()
b=a.encoder_layers().to(DEVICE)
#print(b)
b(fake).shape

In [ ]:
def MSE(fake: torch.Tensor,
        image: torch.Tensor) -> torch.Tensor:
    return torch.mean((fake - image)**2)


def train(model: nn.Sequential,
          train_loader: DataLoader,
          optimizer: torch.optim,
          device: torch.device) -> None:
    for images, _ in train_loader:
        images = images.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        batch_loss = MSE(outputs, images)
        batch_loss.backward()
        optimizer.step()


def test(model: nn.Sequential,
         test_loader: DataLoader,
         device: torch.device) -> float:
    model.eval()
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(DEVICE)
            output = model(images)
            loss = MSE(output, images)
            test_loss += loss.item()
    return test_loss / len(test_loader.dataset)

In [ ]:
epochs = 100
running_loss = 0.0
lr = 0.001
test_loss_list = []
eps = 1e-4

model = Autoencoder().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

t0 = time()

for epoch in range(epochs):
    loss = 0.0
    train(model, train_loader, optimizer, DEVICE)
    test_loss = test(model, train_loader, DEVICE)
    test_loss_list.append(test_loss)
    t1 = (time() - t0) / 60
    print('Epoch: {}, test loss: {:.5f}, '.format(epoch+1, test_loss) + 
            'time: {:.2f} min'.format(t1))
    if epoch > 2:
        if max(test_loss_list[-5:]) - min(test_loss_list[-5:]) > eps:
            continue
        else:
            break
print('Finish!')


In [ ]:
plt.plot(range(1, len(test_loss_list) + 1), test_loss_list)
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import save_image

def imshow(img, im_name):
    img = img.detach().cpu()
    save_image(img, im_name)
    npimg = img.numpy()
    plt.axis('off')
    npimg = np.transpose(npimg, (1, 2, 0))
    #im = Image.fromarray(npimg)
    #im.save(im_name)
    #plt.imsave(im_name, npimg/)
    plt.imshow(npimg)
    plt.show()


def get_file_size_in_bytes(file_path):
   size = os.path.getsize(file_path)
   return size


for i, (im, lab) in enumerate(train_loader):
    print(im.shape)
    print(lab[0])
    imshow(im[0], '/content/image.jpeg')
    fake = model(im.to(DEVICE))
    print(fake.shape)
    imshow(fake[0], '/content/fake.jpeg')
    real_size = get_file_size_in_bytes('/content/image.jpeg')
    compressed_state=model.encoder_layers()
    jpeg_comp_rate = 3*32*32 / real_size
    #ae_comp_rate = sys.getsizeof(fake[0]) / sys.getsizeof(compressed_state(im)[0])
    ae_comp_rate = 3*32*32 / sys.getsizeof(compressed_state(im)[0])
    print('jpeg ompression rate : {:.2f}'.format(jpeg_comp_rate))
    print('AE ompression rate : {:.2f}'.format(ae_comp_rate))
    break

